### Матричные факторизации

В данной работе вам предстоит познакомиться с практической стороной матричных разложений.
Работа поделена на 4 задания:
1. Вам необходимо реализовать SVD разложения используя SGD на explicit данных
2. Вам необходимо реализовать матричное разложения используя ALS на implicit данных
3. Вам необходимо реализовать матричное разложения используя BPR(pair-wise loss) на implicit данных
4. Вам необходимо реализовать матричное разложения используя WARP(list-wise loss) на implicit данных

Мягкий дедлайн 28 Сентября (пишутся замечания, выставляется оценка, есть возможность исправить до жесткого дедлайна)

Жесткий дедлайн 5 Октября (Итоговая проверка)

In [1]:
import implicit
import pandas as pd
import numpy as np
import random
import scipy.sparse as sp
from scipy import stats

from tqdm.autonotebook import tqdm, trange

In [2]:
SEED = 42
if SEED is not None:
    np.random.seed(SEED)
    random.seed(SEED)

В данной работе мы будем работать с explicit датасетом movieLens, в котором представленны пары user_id movie_id и rating выставленный пользователем фильму

Скачать датасет можно по ссылке https://grouplens.org/datasets/movielens/1m/

In [3]:
ratings = pd.read_csv('../ml-1m/ratings.dat', delimiter='::', header=None, 
        names=['user_id', 'movie_id', 'rating', 'timestamp'], 
        usecols=['user_id', 'movie_id', 'rating'], engine='python')

In [4]:
movie_info = pd.read_csv('../ml-1m/movies.dat', delimiter='::', header=None, 
        names=['movie_id', 'name', 'category'], engine='python')

Explicit данные

In [5]:
ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
5,1,1197,3
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4


Для того, чтобы преобразовать текущий датасет в Implicit, давайте считать что позитивная оценка это оценка >=4

In [6]:
implicit_ratings = ratings.loc[(ratings['rating'] >= 4)]

In [7]:
implicit_ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
3,1,3408,4
4,1,2355,5
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4
10,1,595,5
11,1,938,4
12,1,2398,4


Удобнее работать с sparse матричками, давайте преобразуем DataFrame в CSR матрицы

In [8]:
users = implicit_ratings["user_id"]
movies = implicit_ratings["movie_id"]
user_item = sp.coo_matrix((np.ones_like(users), (users, movies)))
user_item_t_csr = user_item.T.tocsr()
user_item_csr = user_item.tocsr()

В качестве примера воспользуемся ALS разложением из библиотеки implicit

Зададим размерность латентного пространства равным 64, это же определяет размер user/item эмбедингов

In [9]:
model = implicit.als.AlternatingLeastSquares(factors=64, iterations=100, calculate_training_loss=True)

В качестве loss здесь всеми любимый RMSE

In [10]:
model.fit(user_item_t_csr)

Построим похожие фильмы по 1 movie_id = Истории игрушек

In [11]:
movie_info.head(5)

,movie_id,name,category
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [13]:
get_similars = lambda item_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                        for x in model.similar_items(item_id)]

Как мы видим, симилары действительно оказались симиларами.

Качество симиларов часто является хорошим способом проверить качество алгоритмов.

P.S. Если хочется поглубже разобраться в том как разные алгоритмы формируют разные латентные пространства, рекомендую загружать полученные вектора в tensorBoard и смотреть на сформированное пространство

In [14]:
get_similars(1, model)

['0    Toy Story (1995)',
 '3045    Toy Story 2 (1999)',
 "2286    Bug's Life, A (1998)",
 '33    Babe (1995)',
 '584    Aladdin (1992)',
 '2315    Babe: Pig in the City (1998)',
 '360    Lion King, The (1994)',
 '1526    Hercules (1997)',
 '2618    Tarzan (1999)',
 '2692    Iron Giant, The (1999)']

Давайте теперь построим рекомендации для юзеров

Как мы видим юзеру нравится фантастика, значит и в рекомендациях ожидаем увидеть фантастику

In [15]:
get_user_history = lambda user_id, implicit_ratings : [movie_info[movie_info["movie_id"] == x]["name"].to_string() 
                                            for x in implicit_ratings[implicit_ratings["user_id"] == user_id]["movie_id"]]

In [16]:
get_user_history(4, implicit_ratings)

['3399    Hustler, The (1961)',
 '2882    Fistful of Dollars, A (1964)',
 '1196    Alien (1979)',
 '1023    Die Hard (1988)',
 '257    Star Wars: Episode IV - A New Hope (1977)',
 '1959    Saving Private Ryan (1998)',
 '476    Jurassic Park (1993)',
 '1180    Raiders of the Lost Ark (1981)',
 '1885    Rocky (1976)',
 '1081    E.T. the Extra-Terrestrial (1982)',
 '3349    Thelma & Louise (1991)',
 '3633    Mad Max (1979)',
 '2297    King Kong (1933)',
 '1366    Jaws (1975)',
 '1183    Good, The Bad and The Ugly, The (1966)',
 '2623    Run Lola Run (Lola rennt) (1998)',
 '2878    Goldfinger (1964)',
 '1220    Terminator, The (1984)']

Получилось! 

Мы действительно порекомендовали пользователю фантастику и боевики, более того встречаются продолжения тех фильмов, которые он высоко оценил

In [17]:
get_recommendations = lambda user_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                               for x in model.recommend(user_id, user_item_csr)]

In [18]:
get_recommendations(4, model)

['585    Terminator 2: Judgment Day (1991)',
 '1271    Indiana Jones and the Last Crusade (1989)',
 '2502    Matrix, The (1999)',
 '1284    Butch Cassidy and the Sundance Kid (1969)',
 '1182    Aliens (1986)',
 '1178    Star Wars: Episode V - The Empire Strikes Back...',
 '1179    Princess Bride, The (1987)',
 '847    Godfather, The (1972)',
 '3402    Close Encounters of the Third Kind (1977)',
 '1892    Rain Man (1988)']

Теперь ваша очередь реализовать самые популярные алгоритмы матричных разложений

Что будет оцениваться:
1. Корректность алгоритма
2. Качество получившихся симиларов
3. Качество итоговых рекомендаций для юзера

In [28]:
import abc
import scipy
from sklearn.metrics.pairwise import cosine_similarity

class Recommender(metaclass=abc.ABCMeta):
    @abc.abstractmethod
    def fit(self, user_items):
        raise NotImplementedError()
    
    def similar_items(self, item_id, k=10):
        assert self.items_embeddings is not None
        similarity = cosine_similarity(self.items_embeddings.T)
        return np.flip(np.argsort(similarity[item_id, :])[-k:]).reshape(-1, 1)
    
    def recommend(self, user_id, user_item_csr, k=10, exclude_viewed=True): 
        assert (self.items_embeddings is not None) and (self.user_embeddings is not None)
        user_embedding = self.user_embeddings[user_id]
        if scipy.sparse.issparse(user_embedding):
            user_embedding = user_embedding.toarray().flatten()
        predicted_rating = user_embedding @ self.items_embeddings
        if hasattr(self, 'items_bias'):
            predicted_rating += self.items_bias
        if not exclude_viewed:
            return np.flip(np.argsort(predicted_rating)[-k:]).reshape(-1, 1)
        user_history = user_item_csr[user_id]
        history_size = user_history.count_nonzero()
        top_predictions = np.argsort(predicted_rating)[-(k + history_size + 1):]
        non_watched = np.setdiff1d(top_predictions, user_history.nonzero()[1])
        return np.flip(non_watched[-k:]).reshape(-1, 1)

In [20]:
N_FACTORS = 64

### Задание 1. Не использую готовые решения, реализовать SVD разложение используя SGD на explicit данных

In [21]:
from sklearn.metrics import mean_squared_error
from sklearn.utils import shuffle

In [22]:
class SVD(Recommender):
    def __init__(self, n_factors, alpha, lr, n_iters, eps=1e-3):
        self.n_factors = n_factors
        self.alpha = alpha
        self.lr = lr
        self.n_iters = n_iters
    
    def fit(self, user_items):
        users_shape, items_shape = user_items.shape
        W = np.random.uniform(0, 1 / np.sqrt(self.n_factors), size=(users_shape, self.n_factors))
        H = np.random.uniform(0, 1 / np.sqrt(self.n_factors), size=(self.n_factors, items_shape))

        W_bias = np.zeros(users_shape) #np.random.normal(loc=3, scale=2, size=users_shape)
        H_bias = np.zeros(items_shape) #np.random.normal(loc=3, scale=2, size=items_shape)
        global_bias = user_item.sum() / user_item.count_nonzero()

        pbar = trange(self.n_iters)
        for epoch in pbar:
            # shuffle train dataset
            rows, columns, data = shuffle(user_items.row, user_items.col, user_items.data)
            for i, j, v in zip(tqdm(rows, leave=False), columns, data):
                error = (W[i, :] @ H[:, j] + W_bias[i] + H_bias[j] + global_bias) - v
                old_w_row = W[i, :].copy()
                W_bias[i] -= self.lr * (error + self.alpha * W_bias[i])
                W[i, :] -= self.lr * (error * H[:, j].T + self.alpha * W[i, :])
                H_bias[j] -= self.lr * (error + self.alpha * H_bias[j])
                H[:, j] -= self.lr * (error * old_w_row.T + self.alpha * H[:, j])
    #             global_bias -= self.lr * (error + self.alpha * global_bias)

            loss = mean_squared_error(data, (W @ H + W_bias.reshape(-1, 1) + H_bias.reshape(1, -1) + global_bias)[rows, columns])
            print(f"\rEpoch: [{epoch + 1}/{self.n_iters}]. mse-loss: {loss}", end='', flush=True)
            pbar.set_postfix(loss=loss)
        
        self.user_embeddings = W
        self.user_bias = W_bias
        self.items_embeddings = H
        self.items_bias = H_bias
        return self

In [23]:
explicit_user_item = sp.coo_matrix((ratings["rating"], (ratings["user_id"], ratings["movie_id"])))

In [24]:
svd = SVD(n_factors=N_FACTORS, alpha=1e-2, lr=0.01, n_iters=10).fit(explicit_user_item)

Epoch: [1/10]. mse-loss: 0.8538753719131159

Epoch: [2/10]. mse-loss: 0.8204182652484981

Epoch: [3/10]. mse-loss: 0.8019257375130544

Epoch: [4/10]. mse-loss: 0.7741693157832767

Epoch: [5/10]. mse-loss: 0.7384106378018636

Epoch: [6/10]. mse-loss: 0.6993138451777199

Epoch: [7/10]. mse-loss: 0.6583900296159153

Epoch: [8/10]. mse-loss: 0.6185070015242412

Epoch: [9/10]. mse-loss: 0.5800275932176713

Epoch: [10/10]. mse-loss: 0.543242069978659


In [25]:
get_similars(1, svd)

['0    Toy Story (1995)',
 '3045    Toy Story 2 (1999)',
 "2286    Bug's Life, A (1998)",
 '584    Aladdin (1992)',
 '591    Beauty and the Beast (1991)',
 '2012    Little Mermaid, The (1989)',
 '2021    Rescuers, The (1977)',
 '360    Lion King, The (1994)',
 '1016    Dumbo (1941)',
 '1838    Mulan (1998)']

In [26]:
get_similars(2628, svd)

['2559    Star Wars: Episode I - The Phantom Menace (1999)',
 '1192    Star Wars: Episode VI - Return of the Jedi (1983)',
 '1178    Star Wars: Episode V - The Empire Strikes Back...',
 '257    Star Wars: Episode IV - A New Hope (1977)',
 '1840    X-Files: Fight the Future, The (1998)',
 '1271    Indiana Jones and the Last Crusade (1989)',
 '1335    Star Trek: First Contact (1996)',
 '3684    Patriot, The (2000)',
 '2324    Star Trek: Insurrection (1998)',
 '325    Star Trek: Generations (1994)']

**в топе хорошие рекомендации: мультфильмы и другие части звездных войн**

In [29]:
get_recommendations(4, svd)

['3401    Dersu Uzala (1974)',
 '3269    For All Mankind (1989)',
 '2953    General, The (1927)',
 '2856    Conformist, The (Il Conformista) (1970)',
 '2836    Sanjuro (1962)',
 '2291    Celebration, The (Festen) (1998)',
 '2282    Nights of Cabiria (Le Notti di Cabiria) (1957)',
 '1950    Seven Samurai (The Magnificent Seven) (Shichin...',
 '1880    Man for All Seasons, A (1966)',
 '1252    Patton (1970)']

### Задание 2. Не использую готовые решения, реализовать матричное разложение используя ALS на implicit данных

In [30]:
import scipy
from scipy import sparse
from scipy.sparse.linalg import spsolve

In [33]:
class ALS(Recommender):
    def __init__(self, alpha, reg, n_factors, n_iters, eps=1e-3):
        self.alpha = alpha
        self.reg = reg
        self.n_factors = n_factors
        self.n_iters = n_iters
        self.eps = eps
        
    def __als_step(self, C, R, X, Y):
        # чтобы не считать на каждой итерации цикла
        YtY = Y.T.dot(Y)
        Y_eye = scipy.sparse.eye(Y.shape[0])
        lambda_eye = self.reg * scipy.sparse.eye(self.n_factors)

        for i in trange(X.shape[0], leave=False):
            # выбираем конкретный пример
            conf_samp = C[i, :].toarray()
            r = R[i, :].toarray()
            
            # уравнение 4 отсюда http://yifanhu.net/PUB/cf.pdf и фикс bottleneck
            cui_loc = scipy.sparse.diags(conf_samp, [0])
            A = YtY + Y.T @ cui_loc @ Y + lambda_eye
            b = Y.T @ (cui_loc + Y_eye) @ r.T 

            X[i] = spsolve(A, b)

        return X
    
    def fit(self, user_items):
        confidence = self.alpha * user_items
        
        users_shape, items_shape = user_items.shape
        X = sparse.csr_matrix(np.random.uniform(0, 1 / np.sqrt(self.n_factors), size=(users_shape, self.n_factors)))
        Y = sparse.csr_matrix(np.random.uniform(0, 1 / np.sqrt(self.n_factors), size=(items_shape, self.n_factors))) 

        for iter_step in trange(self.n_iters): 
            X = self.__als_step(confidence, user_items, X, Y)
            Y = self.__als_step(confidence.T, user_items.T, Y, X)
            
        self.user_embeddings = X
        self.items_embeddings = Y.T
        
        return self

In [36]:
als = ALS(alpha=40, reg=0.1, n_factors=N_FACTORS, n_iters=5).fit(user_item_csr)

In [50]:
get_similars(1, als)

['0    Toy Story (1995)',
 '1245    Groundhog Day (1993)',
 '3045    Toy Story 2 (1999)',
 '33    Babe (1995)',
 '2327    Shakespeare in Love (1998)',
 "2286    Bug's Life, A (1998)",
 '315    Shawshank Redemption, The (1994)',
 '1250    Back to the Future (1985)',
 '1120    Monty Python and the Holy Grail (1974)',
 '589    Silence of the Lambs, The (1991)']

In [51]:
get_similars(2628, als)

['2559    Star Wars: Episode I - The Phantom Menace (1999)',
 '1539    Men in Black (1997)',
 '1335    Star Trek: First Contact (1996)',
 '1192    Star Wars: Episode VI - Return of the Jedi (1983)',
 '1178    Star Wars: Episode V - The Empire Strikes Back...',
 '1081    E.T. the Extra-Terrestrial (1982)',
 '257    Star Wars: Episode IV - A New Hope (1977)',
 '1180    Raiders of the Lost Ark (1981)',
 '770    Independence Day (ID4) (1996)',
 '3106    Galaxy Quest (1999)']

**тоже выглядит неплохо, однако в топе популярное, возможно, bias'ы подправят это**

In [71]:
get_recommendations(4, als)

['3634    Mad Max 2 (a.k.a. The Road Warrior) (1981)',
 '2502    Matrix, The (1999)',
 '2460    Planet of the Apes (1968)',
 '1284    Butch Cassidy and the Sundance Kid (1969)',
 '1271    Indiana Jones and the Last Crusade (1989)',
 '1203    Godfather: Part II, The (1974)',
 '1192    Star Wars: Episode VI - Return of the Jedi (1983)',
 '1182    Aliens (1986)',
 '1178    Star Wars: Episode V - The Empire Strikes Back...',
 '957    African Queen, The (1951)']

**рекомендации для пользователя тоже нормальные: другие части просмотренных фильмов и схожие по жанрам**

### Задание 3. Не использую готовые решения, реализовать матричное разложение BPR на implicit данных

In [156]:
class BPR(Recommender):
    def __init__(self, n_factors, alpha, lr, n_iters, eps=1e-3):
        self.n_factors = n_factors
        self.alpha = alpha
        self.lr = lr
        self.n_iters = n_iters
        
    def fit(self, user_items):
        users_shape, items_shape = user_items.shape
        W = np.random.uniform(0, 1 / np.sqrt(self.n_factors), size=(users_shape, self.n_factors))
        H = np.random.uniform(0, 1 / np.sqrt(self.n_factors), size=(items_shape, self.n_factors))
        
        user_items_coo = user_items.tocoo()
        
        pbar = trange(self.n_iters)
        for epoch in pbar:
            loss = 0
            # shuffle train dataset
            rows, columns, data = shuffle(user_items_coo.row, user_items_coo.col, user_items_coo.data)
            for user_id, positive_item_id, v in zip(tqdm(rows, leave=False), columns, data):
                # find negative for user user_id and positive item positive_item_id with rating v
                negative_item_id = np.random.randint(1, items_shape)
                while user_items[user_id, negative_item_id] > 0:
                    negative_item_id = np.random.randint(1, items_shape)
               
                positive_score = W[user_id] @ H[positive_item_id]
                negative_score = W[user_id] @ H[negative_item_id]                
                difference = positive_score - negative_score # r_uij
                
                sigmoid = np.exp(-difference) / (1. + np.exp(-difference))
                
                old_W_user = W[user_id].copy()
                W[user_id] -= self.lr * (sigmoid * -(H[positive_item_id] - H[negative_item_id]) + self.alpha * W[user_id])
                H[positive_item_id] -= self.lr * (sigmoid * -old_W_user + self.alpha * H[positive_item_id])
                H[negative_item_id] -= self.lr * (sigmoid * old_W_user + self.alpha * H[negative_item_id])
                
                loss += np.log(sigmoid)
                
            loss /= user_items.count_nonzero()
            print(f"Epoch: [{epoch + 1}/{self.n_iters}]. log-loss: {loss}", end='', flush=True)
            pbar.set_postfix(logloss=loss)
            
        
        self.user_embeddings = W
        self.items_embeddings = H.T
        return self

In [157]:
bpr = BPR(n_factors=N_FACTORS, alpha=0.000001, lr=0.07, n_iters=10).fit(user_item_csr)

Epoch: [1/10]. log-los: -1.5498379326244651

Epoch: [2/10]. log-los: -3.0035082529331607

Epoch: [3/10]. log-los: -3.561876817250251

Epoch: [4/10]. log-los: -4.011986683819928

Epoch: [5/10]. log-los: -4.377766545735631

Epoch: [6/10]. log-los: -4.692978435156438

Epoch: [7/10]. log-los: -4.965278385308698

Epoch: [8/10]. log-los: -5.215942611436756

Epoch: [9/10]. log-los: -5.463000226337656

Epoch: [10/10]. log-los: -5.662270931230993


In [206]:
get_similars(2628, bpr)

['2559    Star Wars: Episode I - The Phantom Menace (1999)',
 '257    Star Wars: Episode IV - A New Hope (1977)',
 '1192    Star Wars: Episode VI - Return of the Jedi (1983)',
 '2502    Matrix, The (1999)',
 '1178    Star Wars: Episode V - The Empire Strikes Back...',
 '1180    Raiders of the Lost Ark (1981)',
 '1179    Princess Bride, The (1987)',
 '1959    Saving Private Ryan (1998)',
 '1353    Star Trek: The Wrath of Khan (1982)',
 '1539    Men in Black (1997)']

**выглядит неплохо, в топе другие части и схожие фильмы**

In [208]:
get_recommendations(4, bpr)

['3509    Gladiator (2000)',
 '2789    American Beauty (1999)',
 '2693    Sixth Sense, The (1999)',
 '2502    Matrix, The (1999)',
 '1575    L.A. Confidential (1997)',
 '1284    Butch Cassidy and the Sundance Kid (1969)',
 '1245    Groundhog Day (1993)',
 '1230    Bridge on the River Kwai, The (1957)',
 '1203    Godfather: Part II, The (1974)',
 '1192    Star Wars: Episode VI - Return of the Jedi (1983)']

**вполне неплохо выглядят рекомендации**

### Задание 4. Не использую готовые решения, реализовать матричное разложение WARP на implicit данных

In [198]:
class WARP(Recommender):
    def __init__(self, n_factors, alpha, lr, n_iters, max_negatives_retries=10, margin=1, eps=1e-3):
        self.n_factors = n_factors
        self.alpha = alpha
        self.lr = lr
        self.n_iters = n_iters
        self.max_negatives_retries = max_negatives_retries
        self.margin = margin
        
    def fit(self, user_items):
        users_shape, items_shape = user_items.shape
        W = np.random.uniform(0, 1 / np.sqrt(self.n_factors), size=(users_shape, self.n_factors))
        H = np.random.uniform(0, 1 / np.sqrt(self.n_factors), size=(items_shape, self.n_factors))
        
        user_items_coo = user_items.tocoo()
        
        pbar = trange(self.n_iters)
        for epoch in pbar:
            total_loss = 0
            # shuffle train dataset
            rows, columns, data = shuffle(user_items_coo.row, user_items_coo.col, user_items_coo.data)
            for user_id, positive_item_id, v in zip(tqdm(rows, leave=False), columns, data):
                positive_score = W[user_id] @ H[positive_item_id]
                
                # try to find hard-negative for user user_id and positive item positive_item_id with rating v
                for i in range(1, self.max_negatives_retries + 1):
                    negative_item_id = np.random.randint(1, items_shape)
                    while user_items[user_id, negative_item_id] > 0:
                        negative_item_id = np.random.randint(1, items_shape)
                    negative_score = W[user_id] @ H[negative_item_id] 
                    if self.margin + negative_score - positive_score < 0:
                        # we found hard negative
                        break
                else:
                    # don't found hard negative => won't update embeddings
                    continue
                    
                difference = positive_score - negative_score # r_uij
                
                rank_loss = np.log(self.max_negatives_retries / i) #np.exp(-difference) / (1. + np.exp(-difference))
                loss = rank_loss # * (self.margin + negative_score - positive_score)
                old_W_user = W[user_id].copy()
                
                W[user_id] -= self.lr * (loss * (H[negative_item_id] - H[positive_item_id]) + self.alpha * W[user_id])
                H[positive_item_id] -= self.lr * (loss * -old_W_user + self.alpha * H[positive_item_id])
                H[negative_item_id] -= self.lr * (loss * old_W_user + self.alpha * H[negative_item_id])
                
                total_loss += loss
                
            total_loss /= user_items.count_nonzero()
            print(f"Epoch: [{epoch + 1}/{self.n_iters}]. loss: {total_loss}", end='', flush=True)
            pbar.set_postfix(loss=total_loss)
            
        
        self.user_embeddings = W
        self.items_embeddings = H.T
        return self

In [200]:
warp = WARP(n_factors=N_FACTORS, alpha=0.0000001, lr=0.02, n_iters=6).fit(user_item_csr)

Epoch: [1/6]. loss: 0.5415896516570398

Epoch: [2/6]. loss: 0.4730670769845341

Epoch: [3/6]. loss: 0.45847325294775204

Epoch: [4/6]. loss: 0.4151030536228841

Epoch: [5/6]. loss: 0.37333815559422917

Epoch: [6/6]. loss: 0.33956761233348576


In [202]:
get_similars(2628, warp)

['2559    Star Wars: Episode I - The Phantom Menace (1999)',
 '1192    Star Wars: Episode VI - Return of the Jedi (1983)',
 '1539    Men in Black (1997)',
 '476    Jurassic Park (1993)',
 '376    True Lies (1994)',
 '1271    Indiana Jones and the Last Crusade (1989)',
 '770    Independence Day (ID4) (1996)',
 '585    Terminator 2: Judgment Day (1991)',
 '108    Braveheart (1995)',
 '642    Mission: Impossible (1996)']

**выглядит неплохо, в топе другие части и схожие фильмы**

In [203]:
get_recommendations(4, warp)

['2789    American Beauty (1999)',
 '2502    Matrix, The (1999)',
 '2327    Shakespeare in Love (1998)',
 '1575    L.A. Confidential (1997)',
 '1539    Men in Black (1997)',
 '1250    Back to the Future (1985)',
 '1245    Groundhog Day (1993)',
 '1192    Star Wars: Episode VI - Return of the Jedi (1983)',
 '1179    Princess Bride, The (1987)',
 '1178    Star Wars: Episode V - The Empire Strikes Back...']

**вполне неплохо выглядят рекомендации**